# CellPLATO | Cell Plasticity Analysis Tool

In [ ]:
import cellPLATO as cp

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import imageio

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px

OVERWRITE_DATAFRAMES = True

In [ ]:
# Get the experiment list from the experiments listed in the config 
exp_list = cp.populate_experiment_list()
display(exp_list)
print(cp.SAVED_DATA_PATH)

## Measure shape and motion metrics

In [ ]:
# Load, process and combine the dataframes (including segmentation and migration calculations)
comb_df = cp.combine_dataframes(exp_list)

comb_df, new_factors = cp.measurement_pipeline(comb_df, mixed=cp.MIXED_SCALING, factors_to_timeaverage = cp.ALL_FACTORS) #If AVERAGE_TIME_WINDOWS is true, then the comb df contains new factors, which are output as 'new factors'
display(new_factors)

# Returns a filtered dataframe, while also adding included column to comb_df
comb_df, filt_counts = cp.apply_filters(comb_df)

# Process a time-averaged DataFrame
tavg_df = cp.time_average(comb_df)

### Save newly created dataframes

In [ ]:
OVERWRITE_DATAFRAMES = True


if OVERWRITE_DATAFRAMES:
    comb_df.to_csv(cp.SAVED_DATA_PATH + 'comb_df.csv', index=False)
    tavg_df.to_csv(cp.SAVED_DATA_PATH + 'tavg_df.csv', index=False)

### Option to load dataframes if this step is already complete

In [ ]:
comb_df = pd.read_csv(cp.SAVED_DATA_PATH + 'comb_df.csv')
tavg_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tavg_df.csv')

In [ ]:

tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')

In [ ]:
averaged_values = cp.getaveragevalues(df_in=comb_df, factorstoinclude=cp.ALL_FACTORS)

# Plot all metrics

In [ ]:
# cp.comparative_visualization_pipeline(comb_df, num_factors=cp.ALL_FACTORS) #usually this is done on dr_df so it has all the labels like tsnes and umaps
cp.comparative_visualization_pipeline(comb_df, num_factors=cp.ALL_FACTORS) #usually this is done on dr_df so it has all the labels like tsnes and umaps

# Dimensionality reduction (3D UMAP)

### Perform correlation analysis to understand which factors correlate to one another

In [ ]:
df_in = comb_df
# FACTORSTOINCLUDE = only_tmeans
cp.correlation_matrix_heatmap(df_in, factors = cp.ALL_FACTORS)

### Variance thresholder to decide on input factors for dimensionality reduction

In [ ]:
OKDR_FACTORS = cp.variance_threshold(comb_df, threshold_value=0.03)
# Optionally, define new dr factors here

In [ ]:

REGIONPROPS_LIST = ['area',
                    # 'bbox_area',
                    'eccentricity',
                    'equivalent_diameter',
                    # 'extent',
                    'filled_area',
                    'major_axis_length',
                    # 'minor_axis_length',
                    'orientation',
                    'perimeter',
                    #  'solidity'
                     ]

MIG_FACTORS = ['euclidean_dist',     
                'cumulative_length', 
                'speed',
                # 'orientedness', 
                'directedness',
                'turn_angle',
                'endpoint_dir_ratio',
                'dir_autocorr',
                # 'outreach_ratio',
                'MSD',                
                'max_dist',           
                'glob_turn_deg',
                'arrest_coefficient']

ADDITIONAL_FACTORS = ['aspect', 'rip_L'] # 'rip_p', 'rip_K', 


new_DR_FACTORS = REGIONPROPS_LIST + MIG_FACTORS + ADDITIONAL_FACTORS

In [ ]:
REGIONPROPS_LIST = ['area',
                    'bbox_area',
                    'eccentricity',
                    'equivalent_diameter',
                    'extent',
                    'filled_area',
                    'major_axis_length',
                    'minor_axis_length',
                    'orientation',
                    'perimeter',
                     'solidity'
                     ]

MIG_FACTORS = ['euclidean_dist',     
                'cumulative_length', 
                'speed',
                'orientedness', 
                'directedness',
                'turn_angle',
                'endpoint_dir_ratio',
                'dir_autocorr',
                'outreach_ratio',
                'MSD',                
                'max_dist',           
                'glob_turn_deg',
                'arrest_coefficient']

ADDITIONAL_FACTORS = ['aspect', 'rip_L'] # 'rip_p', 'rip_K', 


new_DR_FACTORS = REGIONPROPS_LIST + MIG_FACTORS + ADDITIONAL_FACTORS

In [ ]:
lab_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'lab_dr_df.csv')
exemplar_df = pd.read_csv(cp.SAVED_DATA_PATH + 'exemplar_df.csv')

### Perform UMAP DR (and tSNE and PCA) and then cluster analysis on the comb_df

In [ ]:
# User can change the umap_nn to achieve good separation of clusters

tsne_perp=150
umap_nn = 80
min_dist = 0.0 
n_components = 3

dr_df = cp.dr_pipeline_multiUMAPandTSNE(comb_df, 
                    dr_factors=new_DR_FACTORS,#can use cp.DR_FACTORS to use the default list
                    n_components = n_components,
                    umap_nn=umap_nn,
                    min_dist= min_dist,
                    scalingmethod = 'choice',) # log2minmax # powertransformer #minmax #standard #robust #choice 

# lab_dr_df, exemplar_df=cp.hdbscan_clustering(dr_df, min_cluster_size=800,min_samples=400,cluster_by='UMAPNDIM',  metric='euclidean', plot=False) # 
lab_dr_df, exemplar_df=cp.hdbscan_clustering(dr_df, min_cluster_size=800,min_samples=400,cluster_by='UMAPNDIM',  metric='euclidean', plot=False) # 
lab_dr_df.name='lab_dr_df'
name = lab_dr_df.name

cp.plot_3D_scatter(lab_dr_df, 'UMAP1', 'UMAP2', 'UMAP3', colorby='label', ticks=False, identifier=name + '_byCLUSTERID___',dotsize = 5, alpha=0.1, markerscale = 18) #color = label or condition   
cp.plot_3D_scatter(lab_dr_df, 'UMAP1', 'UMAP2', 'UMAP3', colorby='condition', ticks=False, identifier=name + '_byCONDITION___',dotsize = 5, alpha=0.1, markerscale = 18) #color = label or condition  

### Write the new dataframes to csv

In [ ]:
OVERWRITE_DATAFRAMES = True

if OVERWRITE_DATAFRAMES:
    dr_df.to_csv(cp.SAVED_DATA_PATH + 'dr_df.csv', index=False)
    lab_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_dr_df.csv', index=False)
    exemplar_df.to_csv(cp.SAVED_DATA_PATH + 'exemplar_df.csv', index=False)

In [ ]:
dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'dr_df.csv')
lab_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'lab_dr_df.csv')
exemplar_df = pd.read_csv(cp.SAVED_DATA_PATH + 'exemplar_df.csv')

## Interactive 3D UMAP plots

In [ ]:
cp.interactive_plot_3D_UMAP(df=lab_dr_df,colorby = 'Condition_shortlabel', symbolby = 'Condition_shortlabel', what = ' AllTimeUMAPwithclusters') # TavgUMAPwithclusters

In [ ]:
df=lab_dr_df

#get unique list of conditions from df
condlist = df['Condition_shortlabel'].unique().tolist()
pickedcondition = condlist[0]

cp.interactive_umap_plot_choosecondition(df=lab_dr_df, condition = pickedcondition)

### Plot out the UMAP subplots coloured by scaled metric and condition

In [ ]:
cp.plot_UMAP_subplots_coloredbymetricsorconditions(df_in=lab_dr_df, x= 'UMAP1', y= 'UMAP2', z = 'UMAP3', n_cols = 5, ticks=False, metrics = cp.ALL_FACTORS, scalingmethod='choice',
                                                   identifier='inferno', colormap='inferno', coloredbycondition = False, samplethedf = False)
cp.plot_UMAP_subplots_coloredbymetricsorconditions(df_in=lab_dr_df, x= 'UMAP1', y= 'UMAP2', z = 'UMAP3', n_cols = 5, ticks=False, metrics = cp.ALL_FACTORS, scalingmethod='choice',
                                                   identifier='inferno', colormap='inferno', coloredbycondition = True, samplethedf = False)

### Perform UMAP DR (and tSNE and PCA) and then cluster analysis on the tavg_df

In [ ]:
tsne_perp=150
umap_nn = 20#4#60
min_dist = 0.0 #0.15 
n_components = 3

tavg_dr_df = cp.dr_pipeline_multiUMAPandTSNE(tavg_df, 
                    dr_factors=new_DR_FACTORS,# new_DR_FACTORS # DR_FACTORS #only_tmeans # cp.DR_FACTORS
                    n_components = n_components,
                    umap_nn=umap_nn,
                    min_dist= min_dist,
                    scalingmethod = 'choice',) 

lab_tavg_dr_df, exemplar_tavg_df=cp.hdbscan_clustering(tavg_dr_df, min_cluster_size=50,min_samples=50,cluster_by='UMAPNDIM',  metric='euclidean', plot=False) # 
lab_tavg_dr_df.name='lab_tavg_dr_df'
name2 = lab_tavg_dr_df.name
cp.plot_3D_scatter(lab_tavg_dr_df, 'UMAP1', 'UMAP2', 'UMAP3', colorby='label', ticks=False, identifier=name2 + '_byCLUSTERID___',dotsize = 30, alpha=0.5, markerscale = 5) #color = label or condition   
cp.plot_3D_scatter(lab_tavg_dr_df, 'UMAP1', 'UMAP2', 'UMAP3', colorby='condition', ticks=False, identifier=name2 + '_byCONDITION___',dotsize = 30, alpha=0.5, markerscale = 5) #color = label or condition  

In [ ]:
OVERWRITE_DATAFRAMES = True

if OVERWRITE_DATAFRAMES:
    tavg_dr_df.to_csv(cp.SAVED_DATA_PATH + 'tavg_dr_df.csv', index=False)
    lab_tavg_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_tavg_dr_df.csv', index=False)
    exemplar_tavg_df.to_csv(cp.SAVED_DATA_PATH + 'exemplar_tavg_df.csv', index=False)

In [ ]:
tavg_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tavg_dr_df.csv')
lab_tavg_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'lab_tavg_dr_df.csv')
exemplar_tavg_df = pd.read_csv(cp.SAVED_DATA_PATH + 'exemplar_tavg_df.csv')

### Add the tavg time averaged cluster labels to the regular df

In [ ]:
#Run this function to put the labels into the lab_tavg_lab_dr_df. Slow function. Can potentially be sped up.

lab_tavg_lab_dr_df=cp.add_tavglabel_todf(lab_dr_df, lab_tavg_dr_df)

In [ ]:
lab_tavg_lab_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_tavg_lab_dr_df.csv', index=False)

In [ ]:
lab_tavg_lab_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'lab_tavg_lab_dr_df.csv')

### Cluster counting - how many cells per cluster ID?

In [ ]:
df=lab_tavg_lab_dr_df
label_counts_df = cp.get_label_counts(df)

cp.plot_label_counts(label_counts_df)

In [ ]:
timeinclus_df=cp.count_time_in_label(tptlabel_dr_df)

# Purity

In [ ]:
cluster_purity_df = cp.purity_pointsinclusterspercondition(lab_dr_df) 
display(cluster_purity_df)
f = cp.purityplot_percentcluspercondition(lab_dr_df, cluster_purity_df) 

In [ ]:
# # Deprecated version

# clust_sum_df = cp.cluster_purity(lab_dr_df)  
# display(clust_sum_df)
# f = cp.purity_plots_dev(lab_dr_df, clust_sum_df)

# Quantify plasticity

In [ ]:
tptlabel_dr_df = cp.count_cluster_changes_with_tavg(lab_tavg_lab_dr_df)

In [ ]:
OVERWRITE_DATAFRAMES = True

if OVERWRITE_DATAFRAMES:
    # ultimate DF with all of the things you want in it
    
    tptlabel_dr_df.to_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')
    # tptlabel_dr_df_compare.to_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df_compare.csv')

In [ ]:
# tptlabel_dr_df
tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv') #spidey

In [ ]:
df=tptlabel_dr_df
# all='\_allcells'
cp.plot_plasticity_changes(df, identifier='\_allcells', maxy=4) #problem with NaNs in the data

In [ ]:
df=tptlabel_dr_df
cp.plot_plasticity_countplots(df, identifier='_allcells')

# Disambiguate the clusters of cells

### both disambiguate_timepoint() and disambiguate_tavg()

In [ ]:
# If you don't have them, load them in here

lab_tavg_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'lab_tavg_dr_df.csv')
exemplar_tavg_df = pd.read_csv(cp.SAVED_DATA_PATH + 'exemplar_tavg_df.csv')
tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')
exemplar_df = pd.read_csv(cp.SAVED_DATA_PATH + 'exemplar_df.csv')

## Take 5 exemplars from each cluster ID

In [ ]:
# Choose a number of exemplars to look at
n=15
exemplar_df = exemplar_df.groupby('label').apply(lambda x: x.sample(min(n,len(x)))).reset_index(drop=True)

In [ ]:
size=220 #

df= tptlabel_dr_df #from the all analysis part
exemp_df=exemplar_df #from the cluster analysis part.

top_dictionary, contributions_df_singletpoints, average_df=cp.contribution_to_clusters_topdictionary(df_in=tptlabel_dr_df,  howmanyfactors=10) #BEFORE disambiguate_tavg(), then: lab_tavg_dr_df BEFORE disambiguate_timepoint(), then: #tptlabel_dr_df 
cp.plot_cluster_averages(top_dictionary, df)
cp.disambiguate_timepoint(df, exemp_df, top_dictionary=top_dictionary, XYRange=size,boxoff=True) 

## This is the version based on finding TIME AVERAGED exemplars
### clusters are defined based on the time averaged data, and found in the not time averaged data

In [ ]:
# Make an exemplars DF that only contains 5 of each thing
tavg_exemplar_df=exemplar_tavg_df
n=15
tavg_exemplar_df = tavg_exemplar_df.groupby('label').apply(lambda x: x.sample(min(n,len(x)))).reset_index(drop=True)

In [ ]:
size=220

df= tptlabel_dr_df #from the all analysis part
exemp_df=tavg_exemplar_df #from the cluster analysis part.

top_dictionary, contributions_df=cp.contribution_to_clusters_topdictionary(df_in=lab_tavg_dr_df,  howmanyfactors=10) #BEFORE disambiguate_tavg(), then: lab_tavg_dr_df BEFORE disambiguate_timepoint(), then: #tptlabel_dr_df 
# plot_cluster_values(top_dictionary, df) For this to work on the wholetrack version, you would have to use the tavg labels not the labels
wholetrack_exemplar_df=cp.disambiguate_tavg(df, exemp_df, top_dictionary=top_dictionary, XYRange=size,boxoff=True)

### Heatmap of metric magnitude (center scaled) per cluster ID

In [ ]:
# This needs to be updated to have the same new scaling as the other plots
cp.clustering_heatmap(df_in=tptlabel_dr_df, dr_factors=DR_FACTORS)

In [ ]:
### Which metrics contributed the most to each cluster?
# Check that this is now redundant with the new function contribution to clusters top dictionary

cp.contribution_to_clusters(df_in=tptlabel_dr_df, threshold_value=0.001) #0.0001

# 